In [ ]:
import json
import pandas as pd
from coinmarketcap import Market
# function that checks for NaN in a pandas df
def check_nan(df):
    count = 0
    for i,j in enumerate(df.isnull().sum()):
        if j != 0:
            print(df.columns[i], "contains {} NaN".format(j))
            count += 1
    if count == 0:
        print("no NaN")


coinmarketcap = Market()
# the output of below call is byte so we need to convert it to str
a = coinmarketcap.ticker()
# conversion bytecode to str
json_thing = a.decode()
json_thing = json.loads(json_thing)
df = pd.DataFrame(json_thing)
df.last_updated = pd.to_datetime(df.last_updated, unit='s')
del df['available_supply']
del df['id']
del df['percent_change_1h']
del df['percent_change_7d']
del df['name']
del df['total_supply']

df = df[['24h_volume_usd', 'symbol', 'last_updated', 'market_cap_usd',
       'percent_change_24h', 'price_btc', 'price_usd', 'rank']]
check_nan(df)
df.dropna(inplace=True)
check_nan(df)
# df.sort_values('24h_volume_usd', ascending=False, inplace=True)
json_thing

In [140]:
# limit = 2000 <> rows 2000
# from 2014-10-11 03:00:00
# to 2015-01-02 10:00:00

print(r.json()['TimeFrom'])
print(r.json()['TimeTo'])

1413021600
1420192800


In [179]:
import datetime
print("Date From",datetime.datetime.fromtimestamp(r.json()['TimeFrom']))
print("Date To", datetime.datetime.fromtimestamp(r.json()['TimeTo']))

date_from  = datetime.datetime(2015, 1, 2, 10, 00, 00).timestamp()
date_from

Date From 2014-10-11 04:00:00
Date To 2015-01-02 10:00:00


1420192800.0

In [170]:
datetime.datetime.fromtimestamp(1420196399)

datetime.datetime(2015, 1, 2, 10, 59, 59)

In [268]:
import datetime
import time
dt = parser.parse("2014-10-11 04:00:00")
time.mktime(dt.timetuple())

1412982000.0

In [160]:
from dateutil import parser
dt = parser.parse("2014-10-11 04:00:00")

### Historical data extraction with help of cryptocompare.com API
https://www.cryptocompare.com/api/#introduction

In [3]:
##### Hourly data from cryprocompare

import pandas as pd
import datetime
import requests
from IPython.display import display

# generic url for data extraction from cryptocompare.com's API
url = 'https://min-api.cryptocompare.com/data/{0}?fsym={1}&tsym={2}&limit={3}&toTs={4}&e={5}'

# parameters for the url
# data resolution {0}
hourly_data = "histohour"
minute_data = "histominute"
# currency from {1}
curr_from = "BTC"
# currency to {2}
curr_to = "EUR"

############################################################
# limit of hours with maximum 2000 (1992 is 83 days) {3}
# calculating and converting time for data extraction {4}
date_from_datetime  = datetime.datetime(2014, 1, 2, 0, 0)
print(date_from_datetime)

# enter date_to that is <= than 83 days from date_from for hourly data or <= 83 hr if minute data
date_to_datetime = datetime.datetime(2014, 3, 3 , 0, 0)
print(date_to_datetime)

# checking that the entered dates have no more than 2000 hr between them
delta_in_hours = int(((date_to_datetime - date_from_datetime).total_seconds() / 3600))
print("There are {0} hrs or {1:.2f} days between the dates {2} and {3}".format(delta_in_hours, delta_in_hours / 24, \
      date_from_datetime, date_to_datetime))
limit_hours = 2000
assert (delta_in_hours < limit_hours), \
"There must be <= {0} hrs between dates, but the current value is {1}. Change the dates interval"\
.format(limit_hours, delta_in_hours)
    
# calculating the the date_to by adding 1999 hr to the date_from
# date_to = (date_from_datetime + datetime.timedelta(hours=2000-1))
date_to_timestamp = date_to_datetime.timestamp()
print("The Date To timestamp is {}".format(date_to_timestamp))
############################################################

# coin exchange name, if not used, then CCAGG  {5}
exchange = "Kraken"#"&e=Kraken" (default exchange is CCCAGG)



r = requests.get(url.format(hourly_data, curr_from, curr_to, delta_in_hours, date_to_timestamp, exchange))

df = pd.DataFrame(r.json()['Data'])
df.time = pd.to_datetime(df.time, unit='s')
if exchange == "": exchange = "CCCAGG"
print("Prices from the exchange: {}".format(exchange))

print("##########################")
df.set_index(df['time'], drop=True, inplace=True)
del df['time']
display(df.head(5))
display(df.tail(5))




2014-01-02 00:00:00
2014-03-03 00:00:00
There are 1440 hrs or 60.00 days between the dates 2014-01-02 00:00:00 and 2014-03-03 00:00:00
The Date To timestamp is 1393804800.0
Prices from the exchange: Kraken
##########################


,close,high,low,open,volumefrom,volumeto
time,,,,,,
2014-01-02 00:00:00,547.90,548.0,546.0,547.00,4.3100,2362.54
2014-01-02 01:00:00,547.00,547.9,546.0,547.90,1.1600,635.53
2014-01-02 02:00:00,546.99,547.0,546.0,547.00,1.8700,1024.30
2014-01-02 03:00:00,546.00,547.0,542.9,546.99,1.5400,840.77
2014-01-02 04:00:00,545.00,546.0,545.0,546.00,0.3693,201.26


,close,high,low,open,volumefrom,volumeto
time,,,,,,
2014-03-02 20:00:00,417.10,417.57,414.20,414.79,13.56,5643.59
2014-03-02 21:00:00,414.23,417.10,414.10,417.10,11.53,4784.49
2014-03-02 22:00:00,415.00,415.87,413.50,414.23,11.02,4572.89
2014-03-02 23:00:00,412.03,415.70,412.03,415.00,16.89,6973.32
2014-03-03 00:00:00,417.90,417.90,412.01,412.03,20.15,8394.93
